In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
from importlib import reload

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [40]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter/special_functions.py'>

### Title

In [41]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        
        # semi-nnlo corrections
        'include_alpha_s' : False,
        'include_log' : False, 
        'include_log2' : False,
        'include_sunset' : False,
        
        # nnnlo corrections
        'include_latt_n3lo' : False,
    }, 
    'fit_type' : 'xpt-ratio', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; experimental: 'simultaneous', 'ma-old'
    'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    
    'use_bijnens_central_value' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'save_results' : False,
}

In [42]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Make bootstrapper
bootstrapper = bs.bootstrapper(
    fit_data, prior=None, order=p_dict['order'], F2=p_dict['F2'],
    include_su2_isospin_corrrection=p_dict['include_su2_isospin_corrrection'], use_bijnens_central_value=p_dict['use_bijnens_central_value'],
    fit_type=p_dict['fit_type'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct']
)

new_prior = bootstrapper.create_prior_from_fit()
data_loader.save_prior(new_prior, bootstrapper.get_name())
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.
{'chiral': 1.0,'spacing_n2lo': 1.0}
{'chiral': 1.05,'spacing_n2lo': 1.0}
{'chiral': 1.0,'spacing_n2lo': 1.05}
{'chiral': 1.0499999999999998,'spacing_n2lo': 1.0499999999999998}
{'chiral': 1.0749999999999997,'spacing_n2lo': 1.0749999999999997}
{'chiral': 1.125,'spacing_n2lo': 1.0249999999999997}
{'chiral': 1.1875,'spacing_n2lo': 1.0124999999999997}
{'chiral': 1.2124999999999997,'spacing_n2lo': 1.0874999999999995}
{'chiral': 1.2937499999999997,'spacing_n2lo': 1.1312499999999992}
{'chiral': 1.40625,'spacing_n2lo': 1.0687499999999992}
{'chiral': 1.5718750000000004,'spacing_n2lo': 1.065624999999999}
{'chiral': 1.678125,'spacing_n2lo': 1.1843749999999984}
{'chiral': 1.9234375000000004,'spacing_n2lo': 1.2703124999999975}
{'chiral': 2.201562500000001,'spacing_n2lo': 1.2046874999999972}
{'chiral': 2.6554687500000016,'spacing_n2lo': 1.2414062499999963}
{'chiral': 3.0070312500000016,'spacing_n2lo': 1.4460937499999948}
{'chiral': 3.7246093750000018,'spacing_n2lo': 1.63632812499

Saving...
Done.

Total time (s):  69.21027779579163 

